In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Data

In [2]:
df = pd.read_csv('data_20180119_20180420.csv', index_col=0)
df.head()

,t0,T1,K1,C1,T2,K2,C2,S0,Adj_S0
0,2018-01-02,2018-01-19,1000.0,191.525,2018-04-20,800.0,396.725000,59.4505,1189.01001
1,2018-01-02,2018-01-19,1100.0,92.525,2018-04-20,1015.0,191.675000,59.4505,1189.01001
2,2018-01-02,2018-01-19,1150.0,46.075,2018-04-20,1025.0,183.025000,59.4505,1189.01001
3,2018-01-02,2018-01-19,1160.0,38.000,2018-04-20,1050.0,157.776341,59.4505,1189.01001
4,2018-01-02,2018-01-19,1170.0,30.450,2018-04-20,1080.0,137.850000,59.4505,1189.01001


In [6]:
t0List = list(df.t0.unique())

# dimension: K[k-th t0][t1 and t2][i-th option with i-th strike]
K = [[np.array(df.loc[df.t0 == t0,'K1']),np.array(df.loc[df.t0==t0,'K2'])] for t0 in t0List]

# dimension: Pi[k-th t0][t1 and t2][i-th option with i-th price]
Pi = [[np.array(df.loc[df.t0 == t0,'C1']),np.array(df.loc[df.t0==t0,'C2'])] for t0 in t0List]

In [4]:
# number of t0
N_t0 = len(t0List)

# number of times (t1, t2)
T = 2

# number of options in hedging
N1, N2 = 20, 20
Size = max(N1, N2)

In [5]:
# Assume marginal distribution is uniform
def margDistr(batch_size):
    data = np.random.uniform(size=(batch_size, T))
    for t in range(T):
        data[:, t] = S_min + (S_max - S_min) * data[:, t] # just a simple test, need further consideration
    return data

In [7]:
# arbitrary
S_max = 2000
S_min = 500
BATCH_SIZE = 100

S = margDistr(BATCH_SIZE)

In [8]:
Phi = (S[:,1]-S[:,0])
# define the payoff function of the derivative

### Variables

In [9]:
# parameter: d
d = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, 1], dtype=tf.float64),
    trainable = True,
    name = 'd'
)

In [10]:
# parameter: lambda
Lambda = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, T, Size], dtype=tf.float64),
    trainable = True,
    name = 'lambda'
)

In [ ]:
# # parameter: Delta
# Delta = tf.Variable(
#     initial_value = tf.random.normal(shape=[N_t0, T-1], dtype=tf.float64),
#     trainable = True,
#     name = 'Delta'
# )

In [11]:
# neural network: Delta(S_1)
Delta = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(BATCH_SIZE,)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(N_t0, activation='linear', dtype=tf.float64)
])

In [23]:
test_model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(BATCH_SIZE,)),
#     tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(N_t0, activation='linear', dtype=tf.float64)
])
test_model(S[:,0].reshape((1,-1)))

<tf.Tensor: shape=(1, 13), dtype=float64, numpy=
array([[-127.81195623,  -19.32590397, -228.56733045,  -42.44770865,
          -8.41811479,  199.92748088,  -72.77604877,   37.43726915,
        -231.08736703,  194.32553725, -216.3646712 ,  -31.54375199,
        -183.32129267]])>

### The step-by-step computation of the objective function

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2}$

In [ ]:
hedgePrice = []
for n in range(N_t0):
    p = d[n]
#     for t in range(T):
#         p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
    hedgePrice.append(p)

In [ ]:
# take a look
hedgePrice

$d+\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^++\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^++\Delta_{1}(S_1)\cdot(S_2-S_1)$

In [ ]:
hedgeTerm = []
for n in range(N_t0):
    p = d[n]
#     for t in range(T):
#         p += tf.reduce_sum(Lambda[n,t,:]*np.maximum(S[:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
#     for t in range(T-1):
#         p += Delta[n,t]*(S[:,t+1]-S[:,t])
    p += Delta(S[:,0].reshape((1,-1)))[0,n]*(S[:,1]-S[:,0])
    hedgeTerm.append(p)

In [ ]:
# take a look
hedgeTerm

$\inf_{h\in\mathcal{H}^m}\int hd\mu_0+\int \beta_\gamma(c-h)d\theta$

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2} + \Gamma\cdot\left[\left(\Phi(S_1,S_2)-d-\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^+-\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^+-\Delta_{1}(S_1)\cdot(S_2-S_1)\right)^+\right]^2$

In [ ]:
# objective function
Price = tf.reduce_mean(hedgePrice, axis=1)

Gamma = 100
diff = Phi - tf.stack(hedgeTerm, axis=0)
Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)

ObjFunc = Price + Gamma*Penalty

In [ ]:
# take a look
Price

In [ ]:
# take a look
Penalty

In [ ]:
# take a look
ObjFunc

### Optimisation

In [12]:
# Combine all the computation as a function
def ObjFunc(d, Delta, K, Pi, Phi, S): # without Lambda
    hedgePrice = []
    for n in range(N_t0):
        p = d[n]
#         for t in range(T):
#             p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
        hedgePrice.append(p)
    
    hedgeTerm = []
    for n in range(N_t0):
        p = d[n]
#         for t in range(T):
#             p += tf.reduce_sum(Lambda[n,t,:]*np.maximum(S[:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
#         for t in range(T-1):
#             p += Delta[n,t]*(S[:,t+1]-S[:,t])
        p += Delta(S[:,0].reshape((1,-1)))[0,n]*(S[:,1]-S[:,0]) # Delta as a neural network
        hedgeTerm.append(p)
    
    Price = tf.reduce_mean(hedgePrice, axis=1)
    Gamma = 100
    diff = Phi-tf.stack(hedgeTerm, axis=0)
    Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)
    
    return Price + Gamma*Penalty, Price

In [13]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = 0.01)

In [101]:
# for step in range(10000):
#     with tf.GradientTape() as tape:
#         obj, price = ObjFunc(d, Lambda, Delta, K, Pi, Phi, S)
#     gradients = tape.gradient(obj, [d, Lambda, Delta])
#     optimizer.apply_gradients(zip(gradients, [d, Lambda, Delta]))

In [14]:
nIter = 10000
for step in range(nIter):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(d)
        tape.watch(Lambda)
        for variable in Delta.trainable_variables:
            tape.watch(variable)

        obj, price = ObjFunc(d, Delta, K, Pi, Phi, S)
    
#     gradients_d, gradients_Lambda = tape.gradient(obj, [d, Lambda])
    gradients_d = tape.gradient(obj, d)
    gradients_Delta = tape.gradient(obj, Delta.trainable_variables)
    optimizer.apply_gradients(
#         zip([gradients_d, gradients_Lambda] + gradients_Delta, [d, Lambda] + Delta.trainable_variables)
        zip([gradients_d] + gradients_Delta, [d] + Delta.trainable_variables)
    )
    
    del tape

In [15]:
price

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([ 2.46947608,  1.84781227, -0.50379744, -0.14651639,  0.3748982 ,
        1.96417373,  1.57579438,  3.51411871,  1.50079181,  0.90772784,
        0.14873744,  1.68885112,  1.02335172])>

In [16]:
obj

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([ 2.47156404,  1.84781501, 24.82670374,  1.99590205,  0.37489831,
        1.96417702,  1.57579478,  3.51455481,  1.50079192,  0.90793362,
        0.14873786,  1.68886959,  1.02335177])>

In [17]:
d

<tf.Variable 'd:0' shape=(13, 1) dtype=float64, numpy=
array([[ 2.46947581],
       [ 1.84781164],
       [-0.50375407],
       [-0.14648237],
       [ 0.37489751],
       [ 1.96417314],
       [ 1.57579365],
       [ 3.51411831],
       [ 1.50079095],
       [ 0.90772754],
       [ 0.1487369 ],
       [ 1.68885061],
       [ 1.02335069]])>

In [18]:
Delta(S[:,0].reshape((1,-1)))

<tf.Tensor: shape=(1, 13), dtype=float64, numpy=
array([[0.99819591, 0.99867338, 0.99996543, 0.99998995, 0.99973086,
        0.9985898 , 0.99886923, 0.99746437, 0.99892325, 0.99933861,
        0.99989285, 0.99878551, 0.99926578]])>